# Test 6

should be answered by building and calibrating a 10-period Black-Derman-Toy model for the short-rate.

Period 1 2 3 4 5 6 7 8 9 10

Spot Rate 3.0% 3.1% 3.2% 3.3% 3.4% 3.5% 3.55% 3.6% 3.65% 3.7%

Assume that $Z^6_0 = \frac{100}{(1+r_6)^6}$


In [41]:
SPOT_RATES = [.03, .031, .032, .033, .034, .035, .0355, .036, .0365, .037]
TEST_RATES = [.073, .0762, .081, .0845, .092, .0964, .1012, .1045, .1122, .1155, .1192, .122, .1232]

In [40]:
from sympy import symbols
from sympy.solvers import solve
from math import exp

class BlackDermanToy():
    """ Black-Derman-Toy Model
    The provided b will be assumed to be the same one for all b_i
    """
    def __init__(self, spot_rates, b=0.05, q=0.5):
        """ initialize the black-derman-toy model
        spot_rates: list of rates to callibrate the model to
        """
        self.spot_rates = spot_rates
        self.a = []
        self.b = b
        self.rates = []
        self.P = [[1]]
        self.q = q

    def calibrate(self):
        """ Iterate from 1 to len(spot_rates) - 1 and and callibrate a_i"""
        a_i = symbols('a_1')
        sr = self.short_rate(a_1, 0)
        ep = self.P[0][0]
        eq = ep/sr - (1/(1+self.spot_rates[0]))
        sol = solve(eq, a_i)
        self.a.append(sol[0] - 1) # have to subtract 1 to get the rate
        
        for i in range(1, len(self.spot_rates)):
            a_i = symbol(f'a_{i+2}')
            diag = 
        return self.a
            
    def short_rate(self, a_i, j):
        """ Get the short rate given some a_i, j.
        Since the a_i is being calibrated for, it is provided .
        Rate is calculated from a_i*exp(b_i*j)
        """
        return a_i*exp(self.b*j)
        
    def get_elementary_price(self, i, j, pe, psr):
        """ Returns a sumpy equation that can then be combined
        with others, and be solved.
        q * previous_elementary_price/(1+prev_short_rate)/100 + 
        (1-q) * previous_elementary_price/(1+prev_short_rate)/100
        """
        eq = q * 100*self.P[i][j]/(1 + prev_short_rate) + (1-q) * 100

In [42]:
bdt = BlackDermanToy(TEST_RATES)
print(bdt.calibrate())



[0.0729999999999995]


In [27]:
from sympy import *
from sympy.abc import k, m, n, x
eq = summation(1/x**k, (k, 3, 10))
eq.expand()

x**(-3) + x**(-4) + x**(-5) + x**(-6) + x**(-7) + x**(-8) + x**(-9) + x**(-10)